### IAPPT
Este archivo contiene la creacion y entrenamiento del modelo que usamos para determinar si la mano esta en posicion de piedra, papel o tijera.
Antes de nada ejecutaremos los imports necesarios.

In [4]:
from IPython.display import clear_output
import tensorflow as tf
import pandas as pd
import os

### Datasets
Primero hemos de entender que datos estamos pasando a nuestro modelo exactamente.
Mediante mediapie obtenemos una serie de puntos llamados landmarks, cada uno con un numero representandolo. 

![alt text](https://google.github.io/mediapipe/images/mobile/hand_landmarks.png)

Para obtener y guardar estos datos en el csv se encarga el ejecutable ```csv_generator.py```.
De todos estos landmarks solo nos interesan nueve: 6,8,10,12,14,16,18,20.
Y quedan definidos asi en el csv:

- gesto: que se representa si esa fila que gesto de PPT representa
- muneca_Y: coordenada Y del landmark 0
- muneca_X: coordenada X del landmark 0
- indicep_Y: coordenada Y del landmark 6
- indicep_X: coordenada X del landmark 6
- indicet_Y: coordenada Y del landmark 8
- indicet_Y: coordenada X del landmark 8
- ...

In [5]:
# A almacenamos los datos de los ficheros
data_train = pd.read_csv('training_data.csv')
data_eval = pd.read_csv('eval_data.csv')


Antes de usar el dataset eliminamos la columna que representa el resultado y la almacenamos en una variable



In [6]:
# Eliminamos el campo con el resultado
train_y = data_train.pop('gesto')
eval_y = data_eval.pop('gesto') 

In [7]:
# Especificamos el nombre de los campos y el tipo de dato
feature_columns = []
for key in data_train.keys():
  feature_columns.append(tf.feature_column.numeric_column(key=key))


### Input funtion
Esta es la funcion que usaremos para alimentar a nuestro modelo con los datos, sus parametros

- ```features``` lista de los landmarks de la mano.

- ```labels``` columna del csv que contiene el gesto que representan los landmarks de features.

- ```training``` en caso de ser True el la funcion devuelve filas seleccionadas de manera aleatoria del dataset.

- ```batch_size``` cantidad de filas que se retunean a la vez al modelo.

In [25]:
def input_fn(features, labels, training=True, batch_size=128):
  # Convertir los inputs a dataset
  dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))

  # Mezclar los datos
  if training:
      dataset = dataset.shuffle(1000).repeat()
  return dataset.batch(batch_size)

In [ ]:
input_fn(data_train, train_y, training=True)

### Creando el modelo
Para este caso lo que necesitamos el un modelo que sepa clasificar las coordenadas de los landmarks en 3 categorias correspondiendo a PPT.
Utilizando un ```DNNClassifier``` o Deep Neural Network Classifier podemos agrupar hacer esto mismo.
Compuesto por 18 inputs, y 2 capas ocultas con 32 y 10 nodos respectivamente, este seria el esquema del modelo. 

![alt text](DNNClassifier.png)

Los parametros son estos:
- ```feature_columns``` lista de los landmarks de la mano en total 18.

- ```hidden_units``` son las dos capas ocultas (el numero de nodos es similar al de la documentacion de TS). 

- ```n_classes``` cantidad de grupos que ha de diferenciar en el dataset (piedra, papel o tijera).

- ```model_dir``` donde queda almacenado el modelo.

- ```optimizer``` los optimizadores son algoritmos usados para disminuir el ```loss``` (errores) del modelo. ```lr``` establece la frecuencia con la que el algoritmo actualiza sus parametros.

In [9]:
classifier = tf.estimator.DNNClassifier(
    feature_columns=feature_columns,
    hidden_units=[32, 10],
    n_classes=3,
    optimizer=tf.keras.optimizers.Adam(lr=0.03),
    model_dir='model'
)
clear_output(False)

### Entrenar el modelo
Aqui es donde empieza a aprender y clasificar.
Para ello se llama con el metodo ```train()``` se invoca a la ```input_fn()```, con los steps definimos el numero de iteraciones del modelo sobre el dataset.

In [10]:
# BORRADO DEL MODELO
for root, dirs, files in os.walk('model'):
    for file in files:
        os.remove(os.path.join(root, file))

In [11]:
classifier.train(input_fn=lambda: input_fn(data_train, train_y, training=True),steps=5000)
clear_output(False)

### Resultados

In [ ]:
# Visualizar la precision del modelo 
eval_result = classifier.evaluate(input_fn=lambda: input_fn(data_eval, eval_y, training=False))
accuracy = float('{accuracy:0.3f}'.format(**eval_result)) * 100
print(f'Precision del modelo: {accuracy}%')